In [1]:
import tkinter as tk
from tkinter import filedialog, scrolledtext
from PIL import Image, ImageTk
import speech_recognition as sr
import google.generativeai as genai
import cv2
import numpy as np
import threading
import requests
import pygame
import os
import serpapi

# Inicializa el reconocimiento de voz
reconocedor = sr.Recognizer()

# Configura el modelo generativo
model_img = genai.GenerativeModel("gemini-1.5-flash")
name = "domain4cookin"
model = genai.GenerativeModel(model_name=f'tunedModels/{name}')
chat = model.start_chat(
    history=[
        {"role": "user", "parts": "el modelo debe actuar como un profesor de culinaria. Recibe una lista de ingredientes y debe proporcionarle al usuario una lista de pasos y guiar al usuario para que efectúe la receta. Solo puede sugerir recetas con los ingredientes que recibe en la lista, únicamente esos."},
        {"role": "model", "parts": "Bien. Dime los ingredientes, y te sugiriré ingrientes, y te daré los pasos, de acuerdo a ellos. Solo los ingredientes que me digas"},
    ]
)

recording = False
audio_thread = None
SERPAPI_KEY= "cb656c3da30661f068548f710b4c3c9d366b03113cfe068670beb752012f01c1"
client = serpapi.Client(api_key=SERPAPI_KEY)
API_KEY = 'sk_7177886669223a38d387fbd8d17d98faefd6ca59c3146d97'  # Reemplaza con tu API Key de Eleven Labs


def sintetizar_voz(texto):
    # Directorio temporal para guardar el archivo de audio
    temp_dir = os.path.join(os.path.expanduser("~"), "AppData", "Local", "Temp")
    audio_path = os.path.join(temp_dir, "respuesta_audio.mp3")

    # Si el archivo ya existe, elimínalo antes de escribir uno nuevo
    if os.path.exists(audio_path):
        os.remove(audio_path)

    url = "https://api.elevenlabs.io/v1/text-to-speech/9BWtsMINqrJLrRacOk9x"  # Cambia YOUR_VOICE_ID por el ID de la voz que quieras usar
    headers = {
        'accept': 'audio/mpeg',
        'xi-api-key': API_KEY,  # Asegúrate de reemplazar con tu API_KEY
        'Content-Type': 'application/json',
    }
    data = {
        "text": texto,
        "model_id": "eleven_multilingual_v2",
        "voice_settings": {
            "stability": 0.5,
            "similarity_boost": 1
        }
    }

    # Solicitud a la API para sintetizar la voz
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        # Guardar el archivo de audio en el directorio temporal
        with open(audio_path, "wb") as f:
            f.write(response.content)
        return audio_path
    else:
        return None


def reproducir_audio(file):
    pygame.mixer.init()  # Inicializa el mezclador de audio
    pygame.mixer.music.load(file)  # Carga el archivo MP3
    pygame.mixer.music.play()  # Reproduce el archivo

    # Mantén el programa en ejecución hasta que el archivo de audio termine
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)  # Pausa el bucle para evitar bloqueo

    pygame.mixer.music.stop()  # Detener la reproducción
    pygame.mixer.quit()  # Cierra el mixer y libera el archivo de audio


def reproducir_audio_thread(file):
    thread = threading.Thread(target=reproducir_audio, args=(file,))
    thread.start()


def reconocer_voz():
    global recording
    if not recording:
        return

    with sr.Microphone() as source:
        reconocedor.energy_threshold = noise_slider.get()
        reconocedor.adjust_for_ambient_noise(source, duration=1)
        audio = reconocedor.listen(source)

        try:
            texto = reconocedor.recognize_google(audio, language="es-ES")
            return texto
        except sr.UnknownValueError:
            return "Could not understand the audio"
        except sr.RequestError as e:
            return f"Error connecting to the speech recognition service; {e}"


def load_image():
    file_path = filedialog.askopenfilename(
        filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.gif")],
        title="Select an image"
    )

    if file_path:
        image = Image.open(file_path)
        return image
    else:
        return None


def capture_from_camera():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        return None
    
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame_rgb)
        return image
    else:
        return None


def send_message():
    texto = input_text.get("1.0", "end-1c")
    if not texto:
        return

    # Envía el mensaje al modelo
    response = chat.send_message(
        texto,
        generation_config=genai.types.GenerationConfig(
            candidate_count=1,
            stop_sequences=["x"],
            max_output_tokens=50,
            temperature=0.7
        )
    )
    
    respuesta_texto = response.text
    
    # Mostrar la respuesta del modelo en el chat
    chat_history.insert(tk.END, f"You: {texto}\n")
    chat_history.insert(tk.END, f"Model: {respuesta_texto}\n")
    input_text.delete("1.0", tk.END)

    # Obtener recetas de SerpAPI basadas en los ingredientes
    recetas = buscar_recetas_en_serpapi(texto)
    if recetas:
        chat_history.insert(tk.END, f"Recetas relacionadas:\n")
        for receta in recetas:
            chat_history.insert(tk.END, f"- {receta['title']}: {receta['link']}\n")
    
    # Generar y reproducir el audio con Eleven Labs
    archivo_audio = sintetizar_voz(respuesta_texto)
    if archivo_audio:
        reproducir_audio_thread(archivo_audio)
def buscar_recetas_en_serpapi(ingredientes):
    try:
        result = client.search(
            q=f"Recetas con {ingredientes}",
            engine="google",
            hl="es",
            gl="co"
        )
        return result.get("recipes_results", [])
    except Exception as e:
        chat_history.insert(tk.END, f"Error al buscar en SerpAPI: {e}\n")
        return []



def handle_audio():
    global audio_thread
    audio_thread = threading.Thread(target=lambda: process_audio())
    audio_thread.start()


def process_audio():
    global recording
    recording = True
    texto = reconocer_voz()
    if texto:
        input_text.delete("1.0", tk.END)
        input_text.insert(tk.END, texto)
        send_message()
    recording = False


def handle_image(image):
    if image:
        try:
            response = model_img.generate_content(
                ["Act as a culinary master and identify each ingredient you see in the image in detail. Be concise and just print the ingredient list. Do something like: The ingredients: (and insert the list)", image],
                generation_config=genai.types.GenerationConfig(
                    candidate_count=1,
                    stop_sequences=["x"],
                    max_output_tokens=50,
                    temperature=0.7
                )
            )
            chat_history.insert(tk.END, f"You: Image loaded\n")
            chat_history.insert(tk.END, f"Model: {response.text}\n")
        except Exception as e:
            chat_history.insert(tk.END, f"Error processing the image: {e}\n")


def on_send_click():
    send_message()


def on_audio_button_press(event):
    handle_audio()


def on_audio_button_release(event):
    global recording
    recording = False


def on_image_click():
    image = load_image()
    if image:
        handle_image(image)
        img_tk = ImageTk.PhotoImage(image)
        image_label.config(image=img_tk)
        image_label.image = img_tk


def on_camera_click():
    image = capture_from_camera()
    if image:
        handle_image(image)
        img_tk = ImageTk.PhotoImage(image)
        camera_label.config(image=img_tk)
        camera_label.image = img_tk


def on_clear_click():
    chat_history.delete(1.0, tk.END)
    image_label.config(image='')
    camera_label.config(image='')


# Configura la ventana de la interfaz gráfica
root = tk.Tk()
root.title("Chat with Culinary Model")

main_frame = tk.Frame(root, padx=10, pady=10)
main_frame.pack(expand=True, fill=tk.BOTH)

left_frame = tk.Frame(main_frame, padx=10, pady=10)
left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

right_frame = tk.Frame(main_frame, padx=10, pady=10)
right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, expand=True)

title_label = tk.Label(right_frame, text="Chat with Culinary Model", font=("Helvetica", 16, "bold"))
title_label.pack(pady=5)

chat_history = scrolledtext.ScrolledText(right_frame, wrap=tk.WORD, height=15, width=40)
chat_history.pack(pady=10, fill=tk.BOTH, expand=True)

input_text = tk.Text(right_frame, height=3, width=40)
input_text.pack(pady=5, fill=tk.BOTH, expand=True)

slider_frame = tk.Frame(right_frame, pady=10)
slider_frame.pack(fill=tk.X)

slider_label = tk.Label(slider_frame, text="Noise Sensitivity:")
slider_label.pack(side=tk.LEFT, padx=5)

noise_slider = tk.Scale(slider_frame, from_=100, to=10000, orient=tk.HORIZONTAL)
noise_slider.set(reconocedor.energy_threshold)
noise_slider.pack(side=tk.LEFT, padx=5, fill=tk.X, expand=True)

button_frame = tk.Frame(right_frame)
button_frame.pack(pady=10)

send_button = tk.Button(button_frame, text="Send", command=on_send_click, width=15)
send_button.grid(row=0, column=0, padx=5)

audio_button = tk.Button(button_frame, text="Voice Recognition", width=20)
audio_button.grid(row=0, column=1, padx=5)
audio_button.bind("<ButtonPress>", on_audio_button_press)
audio_button.bind("<ButtonRelease>", on_audio_button_release)

image_button = tk.Button(button_frame, text="Load Image", command=on_image_click, width=15)
image_button.grid(row=0, column=2, padx=5)

camera_button = tk.Button(button_frame, text="Capture Camera", command=on_camera_click, width=15)
camera_button.grid(row=0, column=3, padx=5)

clear_button = tk.Button(button_frame, text="Clear", command=on_clear_click, width=15)
clear_button.grid(row=0, column=4, padx=5)

image_label = tk.Label(left_frame)
image_label.pack(pady=10)

camera_label = tk.Label(left_frame)
camera_label.pack(pady=10)

# Inicia el bucle principal
root.mainloop()


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
